In [1]:
import random
import numpy as np
import collections

from deap import base, creator, tools, algorithms

In [2]:
#filter_count = [8,16, 32, 64, 128, 256, 384, 512] #number of filters
# mapped values = 000,001,010,011, 100,101,110 and 111
#depth_types = [1,2,3,4]
# mapped values = [0,0][0,1][1,0][1,1] for each of the layers
#kernel = [1,2,3,4] = (3,3),(5,5),(7,7),(9,9)
# mapped values = [0,0][0,1][1,0][1,1]
#optimizer = [1,2,3,4] = 'sgd', 'adam', 'adamax', 'nadam'
# mapped values = [0,0][0,1][1,0][1,1]
#pooling_types = [1,2] for each of the layers
# mapped values = [0][1]

#Individual = [1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0], length=28

#Depth=4, Filters = [64, 16, 64, 512], pooling_type=[1, 1, 4, 1], kernel=[4, 4, 2, 4], optimizer=2
#, fitness=0.033873343151693665

In [3]:
def get_filters(depth):
    # it will be  added later by the creator
    filters = []
    for filter in range(depth):
        filters.append(random.choice(filter_count))
    return filters

def get_pooling_type():
    return random.choice(pooling_types)

In [4]:
compare = lambda x, y: collections.Counter(x) == collections.Counter(y)

def get_unet_params(binary_list):
    d, f, k, o, p = 1, [], [], 1, []
    
    depth_bin = binary_list[0:2]
    if (compare(depth_bin,[0,0])):
        d=1
    elif (compare(depth_bin,[0,1])):
        d=2
    elif (compare(depth_bin,[1,0])):
        d=3
    elif (compare(depth_bin,[1,1])):
        d=4
        
    filter_binary = binary_list[2:14]        
    k_b = binary_list[14:22]
    o_b = binary_list[22:24]
    p_b = binary_list[24:28]

    for i in range(0,d):
        f.append(get_filter_count(filter_binary[i*3:(i*3+3)]))
        k.append(get_bin_count(k_b[i*2:(i*2+2)]))
        p.append(get_bin_count(p_b[i:(i+1)]))
        
    o = get_bin_count(o_b)
    
    return d,f,k,o,p
    
def get_filter_count(cb):
    if (compare(cb,[0,0,0])):
        return 8
    elif (compare(cb,[0,0,1])):
        return 16
    elif (compare(cb,[0,1,0])):
        return 32
    elif (compare(cb,[0,1,1])):
        return 64
    elif (compare(cb,[1,0,0])):
        return 96
    elif (compare(cb,[1,0,1])):
        return 128
    elif (compare(cb,[1,1,0])):
        return 256
    elif (compare(cb,[1,1,1])):
        return 512    
    
def get_bin_count(final):
    if (compare(final,[0,0]) or (sum(final) == 0)):
        return 1
    elif (compare(final,[0,1]) or (sum(final) == 1)):
        return 2
    elif (compare(final,[1,0])):
        return 3
    else:
        return 4
    

In [5]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, typecode="I", fitness=creator.FitnessMax, strategy=None)
creator.create("Strategy", list, typecode="I")

In [6]:
INDIVIDUAL_SIZE = 2 + (4*3) + (4*2) + 2 + (4)

toolbox = base.Toolbox()

def generateES(ind_cls, strg_cls, size):
    ind = ind_cls(random.randint(0,1) for _ in range(size))
    ind.strategy = strg_cls(random.randint(0,1) for _ in range(size))
    return ind

# generation functions
toolbox.register("individual", generateES, creator.Individual, creator.Strategy, INDIVIDUAL_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [7]:
def eval_model_loss_function(individual):
    print(f'individual={individual}')
    d,f,k,o,p = get_unet_params(individual)
    print(f'Depth={d}, Filters = {f}, pooling_type={p}, kernel={k}, optimizer={o}')
    fitness = (sum(f))/(sum(f) + sum(k) + o + sum(p))
    print(f', fitness={fitness}')
    return fitness,

In [8]:
CXPB, MUTPB, NGEN, NPOP = 0.7, 0.1, 5, 10

In [12]:
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)

def selElitistAndTournament(individuals, k, frac_elitist, tournsize):
    return tools.selBest(individuals, int(k*frac_elitist)) + tools.selTournament(individuals, int(k*(1-frac_elitist)), tournsize=tournsize)

toolbox.register("select", selElitistAndTournament, frac_elitist=0.1 , tournsize=3)

toolbox.register("evaluate", eval_model_loss_function)

In [13]:
test_ind = toolbox.individual()
print(test_ind)


[0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]


In [14]:


# initialize parameters
pop = toolbox.population(n=NPOP)
hof = tools.HallOfFame(NPOP * NGEN)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)
stats.register("std", np.std)

# genetic algorithm
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB,
                               ngen=NGEN, stats=stats, halloffame=hof,
                               verbose=True)


individual=[1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1]
Depth=2, Filters = [16, 64], pooling_type=[2, 1], kernel=[2, 2], optimizer=2
, fitness=0.898876404494382
individual=[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1]
Depth=2, Filters = [64, 512], pooling_type=[2, 1], kernel=[2, 4], optimizer=2
, fitness=0.9812606473594548
individual=[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
Depth=2, Filters = [16, 64], pooling_type=[1, 1], kernel=[1, 1], optimizer=2
, fitness=0.9302325581395349
individual=[1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0]
Depth=4, Filters = [64, 64, 64, 64], pooling_type=[1, 2, 1, 1], kernel=[4, 4, 1, 4], optimizer=2
, fitness=0.927536231884058
individual=[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
Depth=1, Filters = [64], pooling_type=[1], kernel=[2], optimizer=1
, fitness=

In [15]:
for cand in hof.items:
    print(cand)
print(logbook)
gen = logbook.select("gen")
fit_maxs = logbook.chapters["fitness"].select("max")
size_avgs = logbook.chapters["size"].select("avg")
print(fit_maxs)
print(size_avgs)
print(hof.items[0])


[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]
[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1]
[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0]
[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]
[1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0